Compare the identified holes found with the older version of VoidFinder (from Jan. 2022) and the latest version of VoidFinder (from ~June 2022).

The files being compared are generated in the `_voidfinder_cython.pyx` files.

In [1]:
import numpy as np

from astropy.table import Table, hstack

# Output files

Read and parse the output files to convert into `astropy` tables (for easier comparisons).

In [2]:
def parse_debug_file(filename):
    '''
    Open and parse the output file, converting its contents into an astropy table.
    
    
    PARAMETERS
    ==========
    
    filename : string
        path to the output file
        
    
    RETURNS
    =======
    
    output_table : astropy table
        Table containing the contents of the given file.  Columns include:
          - "cell" : contains the cell coordinates from which the hole was grown
          - "hole" : contains the center hole coordinates
          - "R_hole" : radius of the hole
          - "gal1" : coodinates of the first bounding galaxy
          - "gal2" : coordinates of the second bounding galaxy
          - "gal3" : coordinates of the third bounding galaxy
          - "gal4" : coordinates of the fourth bounding galaxy
    '''
    
    # Read in file contents
    DEBUG_OUTPUT_FILE = open(filename, 'r')
    DEBUG_OUTPUT = DEBUG_OUTPUT_FILE.read()
    DEBUG_OUTPUT_FILE.close()
    
    # Split up the file contents by new line marker
    DEBUG_OUTPUT_lines = DEBUG_OUTPUT.split('\n')
    
    # Number of cells
    n_cells = int(len(DEBUG_OUTPUT_lines)/7)
    
    output_table = Table()
    output_table['cell'] = np.zeros((n_cells, 3), dtype=int)
    output_table['hole'] = np.zeros((n_cells, 3), dtype=np.float32)
    output_table['R_hole'] = np.zeros(n_cells, dtype=np.float32)
    output_table['gal1'] = np.zeros((n_cells, 3), dtype=np.float32)
    output_table['gal2'] = np.zeros((n_cells, 3), dtype=np.float32)
    output_table['gal3'] = np.zeros((n_cells, 3), dtype=np.float32)
    output_table['gal4'] = np.zeros((n_cells, 3), dtype=np.float32)
    
    for i in range(n_cells):
        
        output_table['cell'][i] = np.fromstring(DEBUG_OUTPUT_lines[7*i][19:-1], dtype=int, sep=', ')
        output_table['hole'][i] = np.fromstring(DEBUG_OUTPUT_lines[7*i+1][14:-1], sep=', ')
        output_table['R_hole'][i] = float(DEBUG_OUTPUT_lines[7*i+2][13:])
        output_table['gal1'][i] = np.fromstring(DEBUG_OUTPUT_lines[7*i+3][12:-1], sep=', ')
        output_table['gal2'][i] = np.fromstring(DEBUG_OUTPUT_lines[7*i+4][12:-1], sep=', ')
        output_table['gal3'][i] = np.fromstring(DEBUG_OUTPUT_lines[7*i+5][12:-1], sep=', ')
        output_table['gal4'][i] = np.fromstring(DEBUG_OUTPUT_lines[7*i+6][12:-1], sep=', ')
        
    return output_table

In [3]:
Jan_output = parse_debug_file('/Users/kellydouglass/Desktop/Voids_old/VAST/VoidFinder/vast/voidfinder/tests/VF_DEBUG_old.txt')

In [4]:
June_output = parse_debug_file('/Users/kellydouglass/Documents/Research/Voids/code/scripts/VF_DEBUG.txt')

In [26]:
linux_output = parse_debug_file('/Users/kellydouglass/Documents/Research/Voids/code/scripts/VF_DEBUG_linux.txt')

# Compare outputs

### Which cells result in a hole in the older version of VF but not the newer one?

In [5]:
keep_rows = []

for i, cell in enumerate(Jan_output['cell']):
    
    if not np.any(np.all(cell == June_output['cell'], axis=1)):
        
        keep_rows.append(i)
        
cells_in_old_not_new = Jan_output[keep_rows]

In [6]:
cells_in_old_not_new

cell,hole,R_hole,gal1,gal2,gal3,gal4
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]
25 .. 5,25.503057 .. 0.12753952,0.770878,26.092567 .. 0.0,24.78794 .. 0.0,24.786995 .. 0.24853383,24.90938 .. 0.0
25 .. 4,25.491405 .. -0.8926407,0.77086836,26.076674 .. -1.046259,24.772839 .. -0.993946,26.083626 .. -0.784764,26.204426 .. -1.046259
25 .. 3,25.448694 .. -1.9117334,0.7708331,26.029007 .. -2.0912433,24.727556 .. -1.9866811,26.0439 .. -1.8301864,26.171494 .. -1.8301864
27 .. 7,26.708588 .. 1.9780577,0.32367545,26.42069 .. 1.8301864,26.405582 .. 2.0912433,26.5271 .. 2.0912433,26.542276 .. 1.8301864
27 .. 4,26.46144 .. -0.9135296,0.1958153,26.453938 .. -1.046259,26.46099 .. -0.784764,26.575676 .. -1.046259,26.582764 .. -0.784764
27 .. 3,26.78695 .. -1.9575716,0.19079724,26.894869 .. -1.8301864,26.87949 .. -2.0912433,26.779373 .. -1.8301864,26.764061 .. -2.0912433
27 .. 4,26.930052 .. -0.9173873,0.19506155,26.928713 .. -1.046259,26.935892 .. -0.784764,26.813072 .. -1.046259,26.820223 .. -0.784764
27 .. 7,26.805674 .. 1.95894,0.18669249,26.87949 .. 2.0912433,26.894869 .. 1.8301864,26.779373 .. 1.8301864,26.764061 .. 2.0912433
27 .. 8,26.619202 .. 2.9885926,0.2368734,26.821077 .. 2.873379,26.797518 .. 3.1336796,26.705898 .. 2.873379,26.682442 .. 3.1336796


### Which cells result in a hole on my Macbook but not on Steve's linux box?

In [27]:
keep_rows = []

for i, cell in enumerate(June_output['cell']):
    
    if not np.any(np.all(cell == linux_output['cell'], axis=1)):
        
        keep_rows.append(i)
        
cells_in_mac_not_linux = June_output[keep_rows]

In [28]:
cells_in_mac_not_linux

cell,hole,R_hole,gal1,gal2,gal3,gal4
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]
8 .. 6,8.544049 .. 1.3138975,0.7518658,7.873082 .. 1.1739212,9.185263 .. 1.3695748,7.863737 .. 1.2516894,9.227532 .. 1.3695748
9 .. 3,9.532032 .. -1.7095493,1.1608478,9.454116 .. -1.7317983,8.436267 .. -1.4843986,8.459867 .. -1.4843986,9.43995 .. -1.8220417
9 .. 3,10.000571 .. -1.7769538,1.5003377,8.575005 .. -1.56175,8.5553255 .. -1.56175,11.407101 .. -2.0823333,11.424218 .. -1.9791981
10 .. 3,10.09472 .. -1.5962033,1.5003415,11.51679 .. -1.8759121,8.637593 .. -1.406934,11.540481 .. -1.8759121,11.53227 .. -1.7724831
10 .. 3,10.197747 .. -1.6527008,1.129539,11.231986 .. -1.8759121,11.247083 .. -1.7724831,9.454116 .. -1.7317983,11.264354 .. -1.8759121
14 .. 7,14.566133 .. 2.2584634,0.75568235,13.840285 .. 2.215604,15.224314 .. 2.4371643,13.886121 .. 2.215604,13.903702 .. 2.0861492
14 .. 2,14.505856 .. -2.6654308,0.7556571,15.159536 .. -2.8632083,13.781397 .. -2.6029167,15.209741 .. -2.8632083,13.847787 .. -2.4739976
19 .. 3,19.409685 .. -1.6757195,0.759776,20.095737 .. -1.8290017,20.145258 .. -1.8290017,18.660328 .. -1.6983587,20.110315 .. -1.6464987
19 .. 8,19.654442 .. 3.2528112,0.7596786,18.927252 .. 3.2161968,20.383194 .. 3.4635966,18.897408 .. 3.2161968,20.412184 .. 3.282846


### And which cells result in a hole in the newer version of VF but not the older one?

In [7]:
keep_rows = []

for i, cell in enumerate(June_output['cell']):
    
    if not np.any(np.all(cell == Jan_output['cell'], axis=1)):
        
        keep_rows.append(i)
        
cells_in_new_not_old = June_output[keep_rows]

In [8]:
cells_in_new_not_old

cell,hole,R_hole,gal1,gal2,gal3,gal4
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]
8 .. 6,8.544049 .. 1.3138975,0.7518658,7.873082 .. 1.1739212,9.185263 .. 1.3695748,7.863737 .. 1.2516894,9.227532 .. 1.3695748
9 .. 3,9.532032 .. -1.7095493,1.1608478,9.454116 .. -1.7317983,8.436267 .. -1.4843986,8.459867 .. -1.4843986,9.43995 .. -1.8220417
9 .. 3,10.000571 .. -1.7769538,1.5003377,8.575005 .. -1.56175,8.5553255 .. -1.56175,11.407101 .. -2.0823333,11.424218 .. -1.9791981
14 .. 7,14.566133 .. 2.2584634,0.75568235,13.840285 .. 2.215604,15.224314 .. 2.4371643,13.886121 .. 2.215604,13.903702 .. 2.0861492
19 .. 3,19.409685 .. -1.6757195,0.759776,20.095737 .. -1.8290017,20.145258 .. -1.8290017,18.660328 .. -1.6983587,20.110315 .. -1.6464987
19 .. 8,19.654442 .. 3.2528112,0.7596786,18.927252 .. 3.2161968,20.383194 .. 3.4635966,18.897408 .. 3.2161968,20.412184 .. 3.282846
20 .. 1,20.401642 .. -3.6808329,0.7612165,21.117975 .. -3.904375,21.149666 .. -3.7109964,19.739687 .. -3.4635966,21.090668 .. -3.7109964
20 .. 7,20.577602 .. 2.3662353,0.76131594,19.90453 .. 2.1935759,21.326283 .. 2.3502598,21.266792 .. 2.3502598,21.30591 .. 2.5453064
20 .. 7,20.551783 .. 2.3277795,0.76300204,19.941515 .. 2.1550343,21.27095 .. 2.2987032,21.252266 .. 2.5069437,19.852089 .. 2.1550343


In [29]:
keep_rows = []

for i, cell in enumerate(linux_output['cell']):
    
    if not np.any(np.all(cell == June_output['cell'], axis=1)):
        
        keep_rows.append(i)
        
cells_in_linux_not_mac = linux_output[keep_rows]

In [30]:
cells_in_linux_not_mac

cell,hole,R_hole,gal1,gal2,gal3,gal4
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]


### For the cells that do result in holes in both versions, are they the same holes (bounded by the same four galaxies)?

##### Old v. new (all on Macbook)

In [9]:
keep_rows = []

for i in range(len(Jan_output)):
    
    matched_row = np.all(Jan_output['cell'][i] == June_output['cell'], axis=1)
    
    if np.any(matched_row):
        
        j = np.argwhere(matched_row)[0][0]
        
        gal1_comp = np.all(Jan_output['gal1'][i] == June_output['gal1'][j])
        gal2_comp = np.all(Jan_output['gal2'][i] == June_output['gal2'][j])
        gal3_comp = np.all(Jan_output['gal3'][i] == June_output['gal3'][j])
        gal4_comp = np.all(Jan_output['gal4'][i] == June_output['gal4'][j])
        
        if not np.all([gal1_comp, gal2_comp, gal3_comp, gal4_comp]):
            
            keep_rows.append([i, j])
            
keep_rows = np.array(keep_rows)
            
diff_gals = hstack([Jan_output[keep_rows[:,0]], June_output[keep_rows[:,1]]])

In [10]:
diff_gals

cell_1,hole_1,R_hole_1,gal1_1,gal2_1,gal3_1,gal4_1,cell_2,hole_2,R_hole_2,gal1_2,gal2_2,gal3_2,gal4_2
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3],int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]
6 .. 5,6.5790586 .. 0.029432198,0.7506349,5.8419447 .. 0.05232291,7.3024306 .. 0.06540364,7.3027086 .. 0.0,7.3171434 .. 0.0,6 .. 5,6.5715446 .. 0.3236309,0.7506336,5.8366065 .. 0.26153487,7.2957582 .. 0.32691857,5.8341603 .. 0.313798,5.848143 .. 0.26153487
6 .. 4,6.5110364 .. -0.96791285,0.75062305,5.785311 .. -0.83445966,7.2316394 .. -1.0430746,7.2224946 .. -1.107802,7.2367706 .. -1.107802,6 .. 4,6.540017 .. -0.7343489,0.7506281,5.810163 .. -0.6267359,7.262704 .. -0.7834199,7.255766 .. -0.84843546,7.277059 .. -0.7834199
7 .. 5,7.4566526 .. 0.035972685,0.7512046,8.151114 .. 0.0,6.7925954 .. 0.0,6.792337 .. 0.06540364,8.117636 .. 0.0,7 .. 5,7.4509745 .. 0.323672,0.7512033,8.14615 .. 0.3138777,6.788458 .. 0.26156476,6.7861304 .. 0.32691857,6.7605753 .. 0.26156476
9 .. 4,9.582186 .. -0.92981595,1.1695617,9.465854 .. -1.0056826,8.464618 .. -0.8620137,8.48973 .. -0.8620137,8.49654 .. -0.7838578,9 .. 4,9.486985 .. -0.7874679,1.0866662,9.582292 .. -0.27466738,9.538881 .. -0.457686,8.4517565 .. -0.7056423,8.477552 .. -0.7056423
9 .. 5,9.498186 .. -0.042855795,0.83021754,10.148833 .. 0.457686,8.687305 .. 0.0,8.686975 .. -0.078484364,8.707288 .. 0.0,9 .. 5,9.418173 .. 0.21300372,0.77264565,10.148833 .. 0.457686,8.684328 .. 0.2354292,8.685982 .. 0.15696275,8.665341 .. 0.15696275
9 .. 4,9.90121 .. -1.0309644,1.2979801,8.667201 .. -0.8620137,8.64731 .. -0.8620137,10.332918 .. -0.091565095,8.639715 .. -0.9401039,9 .. 4,9.770767 .. -0.7508294,1.1393452,8.686077 .. -0.627373,8.666143 .. -0.627373,10.3170185 .. 0.0,8.660525 .. -0.7056423
10 .. 3,10.066066 .. -1.7769538,1.5003377,11.483202 .. -2.0823333,8.612402 .. -1.56175,11.506823 .. -2.0823333,8.625326 .. -1.4843986,10 .. 3,10.09472 .. -1.5962033,1.5003415,11.51679 .. -1.8759121,8.637593 .. -1.406934,11.540481 .. -1.8759121,11.53227 .. -1.7724831
10 .. 4,10.527504 .. -0.9937879,1.1328816,10.332918 .. -0.091565095,11.615978 .. -1.0451437,11.606667 .. -1.1493516,11.639872 .. -1.0451437,10 .. 4,10.458529 .. -0.8966166,1.1660626,10.332918 .. -0.091565095,11.599607 .. -0.9408564,11.573928 .. -0.9408564,11.591199 .. -1.0451437
10 .. 4,10.147962 .. -0.93773896,1.2661263,11.32872 .. -1.0451437,9.538881 .. -0.457686,9.473447 .. -0.91450083,11.361367 .. -1.0451437,10 .. 4,10.260826 .. -0.70814395,1.1663233,11.350782 .. -0.73207456,9.69364 .. 0.091565095,11.344293 .. -0.8364973,11.383493 .. -0.73207456


##### Mac v. linux (both new versions)

In [31]:
keep_rows = []

for i in range(len(linux_output)):
    
    matched_row = np.all(linux_output['cell'][i] == June_output['cell'], axis=1)
    
    if np.any(matched_row):
        
        j = np.argwhere(matched_row)[0][0]
        
        gal1_comp = np.all(linux_output['gal1'][i] == June_output['gal1'][j])
        gal2_comp = np.all(linux_output['gal2'][i] == June_output['gal2'][j])
        gal3_comp = np.all(linux_output['gal3'][i] == June_output['gal3'][j])
        gal4_comp = np.all(linux_output['gal4'][i] == June_output['gal4'][j])
        
        if not np.all([gal1_comp, gal2_comp, gal3_comp, gal4_comp]):
            
            keep_rows.append([i, j])
            
keep_rows = np.array(keep_rows)
            
diff_gals_linux = hstack([linux_output[keep_rows[:,0]], June_output[keep_rows[:,1]]])

In [32]:
diff_gals_linux

cell_1,hole_1,R_hole_1,gal1_1,gal2_1,gal3_1,gal4_1,cell_2,hole_2,R_hole_2,gal1_2,gal2_2,gal3_2,gal4_2
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3],int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]
6 .. 5,6.5790586 .. 0.029432198,0.7506349,5.8419447 .. 0.05232291,7.3024306 .. 0.06540364,7.3027086 .. 0.0,7.3171434 .. 0.0,6 .. 5,6.5715446 .. 0.3236309,0.7506336,5.8366065 .. 0.26153487,7.2957582 .. 0.32691857,5.8341603 .. 0.313798,5.848143 .. 0.26153487
6 .. 4,6.5110364 .. -0.96791285,0.75062305,5.785311 .. -0.83445966,7.2316394 .. -1.0430746,7.2224946 .. -1.107802,7.2367706 .. -1.107802,6 .. 4,6.540017 .. -0.7343489,0.7506281,5.810163 .. -0.6267359,7.262704 .. -0.7834199,7.255766 .. -0.84843546,7.277059 .. -0.7834199
7 .. 5,7.4566526 .. 0.035972685,0.7512046,8.151114 .. 0.0,6.7925954 .. 0.0,6.792337 .. 0.06540364,8.117636 .. 0.0,7 .. 5,7.4509745 .. 0.323672,0.7512033,8.14615 .. 0.3138777,6.788458 .. 0.26156476,6.7861304 .. 0.32691857,6.7605753 .. 0.26156476
9 .. 4,9.582186 .. -0.92981595,1.1695617,9.465854 .. -1.0056826,8.464618 .. -0.8620137,8.48973 .. -0.8620137,8.49654 .. -0.7838578,9 .. 4,9.486985 .. -0.7874679,1.0866662,9.582292 .. -0.27466738,9.538881 .. -0.457686,8.4517565 .. -0.7056423,8.477552 .. -0.7056423
9 .. 5,9.481092 .. -0.042878814,0.8354817,10.148833 .. 0.457686,8.687305 .. 0.0,8.686975 .. -0.078484364,8.666661 .. 0.0,9 .. 5,9.418173 .. 0.21300372,0.77264565,10.148833 .. 0.457686,8.684328 .. 0.2354292,8.685982 .. 0.15696275,8.665341 .. 0.15696275
9 .. 4,9.90121 .. -1.0309644,1.2979801,8.667201 .. -0.8620137,8.64731 .. -0.8620137,10.332918 .. -0.091565095,8.639715 .. -0.9401039,9 .. 4,9.770767 .. -0.7508294,1.1393452,8.686077 .. -0.627373,8.666143 .. -0.627373,10.3170185 .. 0.0,8.660525 .. -0.7056423
10 .. 4,10.4679165 .. -0.9902354,1.1716266,10.332918 .. -0.091565095,11.615978 .. -1.0451437,11.606667 .. -1.1493516,11.581908 .. -1.1493516,10 .. 4,10.458529 .. -0.8966166,1.1660626,10.332918 .. -0.091565095,11.599607 .. -0.9408564,11.573928 .. -0.9408564,11.591199 .. -1.0451437
10 .. 4,10.147962 .. -0.93773896,1.2661263,11.32872 .. -1.0451437,9.538881 .. -0.457686,9.473447 .. -0.91450083,11.361367 .. -1.0451437,10 .. 4,10.260826 .. -0.70814395,1.1663233,11.350782 .. -0.73207456,9.69364 .. 0.091565095,11.344293 .. -0.8364973,11.383493 .. -0.73207456
11 .. 4,11.444602 .. -0.9442371,0.7535813,12.138917 .. -1.0584635,10.790149 .. -0.9408564,12.146791 .. -0.94105947,12.09577 .. -0.94105947,11 .. 4,11.457702 .. -0.72233874,0.753586,12.159766 .. -0.70604557,10.808681 .. -0.627596,12.153742 .. -0.82358384,12.108689 .. -0.70604557


##### Old version on Mac v. new version on linux

In [39]:
keep_rows = []

for i in range(len(linux_output)):
    
    matched_row = np.all(linux_output['cell'][i] == Jan_output['cell'], axis=1)
    
    if np.any(matched_row):
        
        j = np.argwhere(matched_row)[0][0]
        
        gal1_comp = np.all(linux_output['gal1'][i] == Jan_output['gal1'][j])
        gal2_comp = np.all(linux_output['gal2'][i] == Jan_output['gal2'][j])
        gal3_comp = np.all(linux_output['gal3'][i] == Jan_output['gal3'][j])
        gal4_comp = np.all(linux_output['gal4'][i] == Jan_output['gal4'][j])
        
        if not np.all([gal1_comp, gal2_comp, gal3_comp, gal4_comp]):
            
            keep_rows.append([i, j])
            
keep_rows = np.array(keep_rows)
            
diff_gals_oldMac_newLinux = hstack([linux_output[keep_rows[:,0]], Jan_output[keep_rows[:,1]]])

In [40]:
diff_gals_oldMac_newLinux

cell_1,hole_1,R_hole_1,gal1_1,gal2_1,gal3_1,gal4_1,cell_2,hole_2,R_hole_2,gal1_2,gal2_2,gal3_2,gal4_2
int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3],int64[3],float32[3],float32,float32[3],float32[3],float32[3],float32[3]
9 .. 5,9.481092 .. -0.042878814,0.8354817,10.148833 .. 0.457686,8.687305 .. 0.0,8.686975 .. -0.078484364,8.666661 .. 0.0,9 .. 5,9.498186 .. -0.042855795,0.83021754,10.148833 .. 0.457686,8.687305 .. 0.0,8.686975 .. -0.078484364,8.707288 .. 0.0
10 .. 4,10.4679165 .. -0.9902354,1.1716266,10.332918 .. -0.091565095,11.615978 .. -1.0451437,11.606667 .. -1.1493516,11.581908 .. -1.1493516,10 .. 4,10.527504 .. -0.9937879,1.1328816,10.332918 .. -0.091565095,11.615978 .. -1.0451437,11.606667 .. -1.1493516,11.639872 .. -1.0451437
22 .. 6,22.470993 .. 0.51628363,1.1909372,22.688585 .. 1.6729946,21.315182 .. 0.588573,21.31924 .. 0.39240688,22.753967 .. 1.6729946,22 .. 6,22.454826 .. 0.51628363,1.1876057,22.688585 .. 1.6729946,21.315182 .. 0.588573,21.31924 .. 0.39240688,22.43376 .. 1.255192
23 .. 6,24.131496 .. 0.18318376,2.038927,23.795002 .. 1.9993198,22.43376 .. 1.255192,23.520187 .. 1.1115232,23.455706 .. 0.22237237,23 .. 6,23.643204 .. 0.7663036,1.3082589,23.795002 .. 1.9993198,22.43376 .. 1.255192,23.520187 .. 1.1115232,23.582727 .. 1.5556585
23 .. 5,23.885582 .. 0.090109594,1.8399919,22.236984 .. 0.0,23.455706 .. 0.22237237,22.306892 .. -0.6278112,22.369085 .. 1.0461395,23 .. 5,23.720175 .. 3.8857806e-16,1.5940763,22.236984 .. 0.0,23.455706 .. 0.22237237,22.306892 .. -0.6278112,23.455706 .. -0.22237237
23 .. 7,23.582716 .. 1.8400773,0.85963666,22.802826 .. 1.8817128,24.191034 .. 2.442372,24.123552 .. 2.442372,22.753967 .. 1.6729946,23 .. 7,23.505428 .. 2.0507567,0.7962911,22.802826 .. 1.8817128,24.191034 .. 2.442372,24.123552 .. 2.442372,22.786297 .. 2.0902874
24 .. 5,24.63129 .. -6.466651e-17,2.0783286,24.874498 .. 0.4447278,24.874498 .. -0.4447278,23.219433 .. 0.0,23.158209 .. -0.6278112,24 .. 5,24.222895 .. -6.466651e-17,1.0141531,24.874498 .. 0.4447278,24.874498 .. -0.4447278,23.219433 .. 0.0,23.270952 .. 0.0
24 .. 6,24.977581 .. 0.11823312,2.397737,24.401966 .. 2.2209306,23.929287 .. 2.2209306,23.795002 .. 1.9993198,25.77174 .. 2.3515735,24 .. 6,24.543713 .. 0.11617938,2.2778404,24.401966 .. 2.2209306,23.929287 .. 2.2209306,23.795002 .. 1.9993198,23.152035 .. 0.83700716
24 .. 4,24.801394 .. -0.34045178,1.9989504,24.982323 .. -1.4120911,23.520187 .. -1.1115232,24.912268 .. -0.9416331,23.795002 .. -1.9993198,24 .. 4,24.34806 .. -1.0298654,0.83239645,24.982323 .. -1.4120911,23.520187 .. -1.1115232,24.912268 .. -0.9416331,24.903727 .. -1.176907


#### Which version of VF is correct?

That is, which version is actually finding the right galaxies?

In [11]:
coord_min = np.array([0., 0., -5.46612512]) # Copied from terminal output

dl = 1.

In [41]:
hole_center_start = (diff_gals['cell_1'] + 0.5)*dl + coord_min
hole_center_start_linux = (diff_gals_linux['cell_1'] + 0.5)*dl + coord_min
hole_center_start_oldMac_newLinux = (diff_gals_oldMac_newLinux['cell_1'] + 0.5)*dl + coord_min

##### Distance between initial hole center and first galaxy found

In [13]:
Jan_gal1_dist2 = np.sum((hole_center_start - diff_gals['gal1_1'])**2, axis=1)
June_gal1_dist2 = np.sum((hole_center_start - diff_gals['gal1_2'])**2, axis=1)

In [36]:
mac_gal1_dist2 = np.sum((hole_center_start_linux - diff_gals_linux['gal1_2'])**2, axis=1)
linux_gal1_dist2 = np.sum((hole_center_start_linux - diff_gals_linux['gal1_1'])**2, axis=1)

In [42]:
oldMac_gal1_dist2 = np.sum((hole_center_start_oldMac_newLinux - diff_gals_oldMac_newLinux['gal1_2'])**2, axis=1)
newLinux_gal1_dist2 = np.sum((hole_center_start_oldMac_newLinux - diff_gals_oldMac_newLinux['gal1_1'])**2, axis=1)

In [44]:
#Jan_gal1_dist2 <= June_gal1_dist2
#linux_gal1_dist2 <= mac_gal1_dist2
oldMac_gal1_dist2 <= newLinux_gal1_dist2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

So January's version of VF always finds a closer galaxy.  And running the new code on Linux (Steve's machine) also finds a closer galaxy than when I run it on my Macbook.  :(

In [16]:
diff_gals['cell_1'][0]

array([6, 1, 5])

In [17]:
diff_gals['gal1_1'][0]

array([5.8419447 , 1.3487191 , 0.05232291], dtype=float32)

In [18]:
diff_gals['gal1_2'][0]

array([5.8366065 , 1.3474869 , 0.26153487], dtype=float32)

In [19]:
diff_gals['hole_1'][0]

array([6.5790586, 1.488689 , 0.0294322], dtype=float32)

In [20]:
diff_gals['hole_2'][0]

array([6.5715446, 1.4869888, 0.3236309], dtype=float32)

In [21]:
diff_gals['R_hole_1'][0]

0.7506349

In [22]:
diff_gals['R_hole_2'][0]

0.7506336

In [23]:
hole_center_start[0]

array([6.5       , 1.5       , 0.03387488])